# Data Ingestion

Data Sources: <br>
<ul>
    <li>SP500 Data: Yahoo! Finance</li>
    <li>Federal Funds Data: FRED</li>
    <li>Industrial Production Data: FRED</li>
    <li>Value/Momentum Data: AQR</li>
</ul>

**Goal**: The purpose of this notebook is to curate one dataset that has each of the datum above as a column

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os

In [2]:
# Read in individual data sources
sp500 = pd.read_csv("./DataSrcs/sp500_data.csv")
fed   = pd.read_csv("./DataSrcs/FEDFUNDS.csv")
ipi   = pd.read_csv("./DataSrcs/INDPRO.csv")
vmd   = pd.read_csv("./DataSrcs/value_momentum_data.csv")

## Update - 12/4/2018, fetch FF data
ff = pd.read_csv("./DataSrcs/ff_data.csv")
ff.Date = ff.Date.apply( lambda x: datetime.strptime(str(x), "%Y%m").date() )
ffmom = pd.read_csv("./DataSrcs/ff_mom_data.csv")
ffmom.Date = ffmom.Date.apply( lambda x: datetime.strptime(str(x), "%Y%m").date() )

In [3]:
date_series = pd.Series( index = pd.RangeIndex(0, len(vmd.Date)) )
for idx, date in enumerate(vmd.Date):
    L = date.split("/"); m = L[0]; d = L[1]; year = L[2]
    
    # check the year
    if int(year) >= 72:
        year = "19" + year
    else:
        year = "20" + year
        
    date_series[idx] = datetime(int(year), int(m), 1).date()
    
vmd.Date = date_series

## Homogenizing

Let's make all of the data homogenous - i.e. same data format to groupby. Also over the same time range. To that end, we'll look from February 1972 - September 2018 and we're going to remove each of the "days" once we convert the date columns to datetime

In [4]:
start_date = datetime(1972, 2, 1); end_date = datetime(2012, 9, 1)

In [5]:
all_data = [sp500, fed, ipi, vmd, ff, ffmom]
for df in all_data:
    dates = df.iloc[:,0]

In [6]:
# Cast to datetime
sp500.Date = pd.to_datetime(sp500.Date)
fed.DATE   = pd.to_datetime(fed.DATE)
ipi.DATE   = pd.to_datetime(ipi.DATE)
vmd.Date   = pd.to_datetime(vmd.Date)
ff.Date    = pd.to_datetime(ff.Date)
ffmom.Date = pd.to_datetime(ffmom.Date)

# Filter so that it's in between Feb72 - Sep18
sp500  = sp500[ (start_date <= sp500.Date) & (sp500.Date <= end_date) ]
fed    = fed[ (start_date <= fed.DATE) & (fed.DATE <= end_date) ]
ipi    = ipi[ (start_date <= ipi.DATE) & (ipi.DATE <= end_date) ]
vmd    = vmd[ (start_date <= vmd.Date) & (vmd.Date <= end_date) ]
ff     = ff[ (start_date <= ff.Date) & (ff.Date <= end_date) ]
ffmom  = ffmom[ (start_date <= ffmom.Date) & (ffmom.Date <= end_date) ]

# Redefine with the new dataframes
all_data = [sp500, fed, ipi, vmd, ff, ffmom]

In [7]:
# Merge all individual dataframes
dataset = pd.concat(all_data, axis=1)
dataset.index = dataset.iloc[:,0]
dataset = dataset.drop(["Date", "DATE", "DATE", "Date"], axis=1)
dataset["VAL^US"] = dataset["VAL^US"].shift(-1)
dataset["MOM^US"] = dataset["MOM^US"].shift(-1)

# Drop unneeded columns
dataset = dataset.drop( ["Open", "High", "Low", "Close", "HML", "RMW", "CMA", "RF"], axis=1 )
#dataset["Mkt-RF"] /= 100; dataset["SMB"] /= 100; dataset.iloc[:,8] /= 100

# Convert pcts to decimals, strings to floats

In [8]:
dataset["VAL^US"].apply( lambda x: float(str(x)[:-1]) if str(x) != "na" else np.na )

ValueError: could not convert string to float: 'na'

In [ ]:
# Save as csv file for you all to just pull from the repo
# It will easily create the data set as "dataset_{}" where {} is the date you ran this code
td = str(datetime.today().date())
dataset.to_csv( "./all_dataset_{}.csv".format(td, td) )

## Build an "AQR" dataset
aqr_dataset = ff_dataset = dataset.iloc[:,[0,1,2,3,4,5]]
aqr_dataset.to_csv( "./aqr_dataset_{}.csv".format(td, td) )

## Build an "FF" dataset
ff_dataset  = dataset.iloc[:,[1,3,6,7,8]]
ff_dataset.to_csv( "./ff_dataset_{}.csv".format(td, td) )

# Put all in the correct directory
os.system( "mkdir '{}_Data'".format(td) )
os.system( "mv 'aqr_dataset_{}.csv' 'ff_dataset_{}.csv' \
'all_dataset_{}.csv' './{}_Data'".format(td, td, td, td) )